In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
%run ../conf.py
%run ../run_xgboost.py

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 2500)

In [3]:
# For loading bigtrain/bigtest for manual inspection
bigtrain = pd.read_csv(conf.basepath+'fulltrain.csv', dtype=pickle.load(open(conf.dtypesf, 'rb')))
#bigtest = pd.read_csv(conf.basepath+'test.csv', dtype=pickle.load(open(conf.dtypesf, 'rb')))
print(len(bigtrain))

145231


In [8]:
bigtrain.VAR_0493.describe()

count     145231
unique       608
top             
freq      136100
Name: VAR_0493, dtype: object

In [4]:
from nltk import word_tokenize
from nltk import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline


class PorterStemmerTokenizer(object):
    """A tokenizer that also stems tokens using a porter stemmer"""
    def __init__(self):
        self.non_alphanum_regex = re.compile('[^ 0-9a-zA-Z]')
        self.porter = PorterStemmer()
    def __call__(self, doc):
        doc = self.non_alphanum_regex.sub(' ', doc)
        tokens_alpha = word_tokenize(doc.lower())
        return [self.porter.stem(t) for t in tokens_alpha]

In [14]:
vec = CountVectorizer(tokenizer=PorterStemmerTokenizer(), 
                      min_df=10, 
                      ngram_range=(1, 3),
                      binary=True
                     )

In [ ]:
vec.fit_transform(['a cat jumped', 'a rat jumps']).toarray()

In [7]:
bigtrain.loc[bigtrain.VAR_0404=='-1', 'VAR_0404'] = ' '
bigtrain.loc[bigtrain.VAR_0493=='-1', 'VAR_0493'] = ' '
bigtrain.VAR_0404.fillna(' ', inplace=True)
bigtrain.VAR_0493.fillna(' ', inplace=True)
bigtrain['COMBINED_TITLE'] = bigtrain['VAR_0404']+' '+bigtrain['VAR_0493']

In [15]:
vec.fit_transform(bigtrain.COMBINED_TITLE)

<145231x601 sparse matrix of type '<class 'numpy.int64'>'
	with 59278 stored elements in Compressed Sparse Row format>

In [17]:
vec.get_feature_names()

['3',
 'account',
 'account execut',
 'account manag',
 'activ',
 'address',
 'address contact',
 'administr',
 'administr assist',
 'admiss',
 'advanc',
 'advisor',
 'agent',
 'agent real',
 'agent real estat',
 'agent student',
 'aid',
 'air',
 'all',
 'all offic',
 'analyst',
 'and',
 'and famili',
 'and famili therapist',
 'and investig',
 'and investig servic',
 'and surgeon',
 'applic',
 'apprais',
 'apprentic',
 'arm',
 'arm secur',
 'arm secur guard',
 'assisit',
 'assist',
 'associ',
 'associ manag',
 'athlet',
 'athlet trainer',
 'attorney',
 'author',
 'author employe',
 'author repres',
 'barber',
 'barber cosmetolog',
 'barber cosmetologist',
 'baton',
 'baton cert',
 'baton cert use',
 'baton certif',
 'baton certif side',
 'board',
 'board member',
 'board of',
 'board of truste',
 'branch',
 'broker',
 'broker individu',
 'broker or',
 'broker or sale',
 'builder',
 'builder individu',
 'bureau',
 'bureau of',
 'bureau of secur',
 'busi',
 'busi offic',
 'busi offic man

In [ ]:
bigtrain.loc[(bigtrain.VAR_0404!='-1')&(bigtrain.VAR_0493!='-1'), ['COMBINED_TITLE', 'VAR_0404', 'VAR_0493']]